# CS181 Practical I

In [5]:
import numpy as np
import matplotlib.pyplot as plt

## Load the data files into local memory from the GCP bucket.

In [23]:
!gsutil cp gs://cs181_practical_data/Xtrain_amp.npy ./
!gsutil cp gs://cs181_practical_data/ytrain_amp.npy ./

!gsutil cp gs://cs181_practical_data/Xtest_amp.npy ./
!gsutil cp gs://cs181_practical_data/ytest_amp.npy ./

!gsutil cp gs://cs181_practical_data/Xtrain_mel.npy ./
!gsutil cp gs://cs181_practical_data/ytrain_mel.npy ./

!gsutil cp gs://cs181_practical_data/Xtest_mel.npy ./
!gsutil cp gs://cs181_practical_data/ytest_mel.npy ./

Copying gs://cs181_practical_data/Xtrain_amp.npy...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

/ [1 files][934.2 MiB/934.2 MiB]   35.9 MiB/s                                   
Operation completed over 1 objects/934.2 MiB.                                    
Copying gs://cs181_practical_data/ytrain_amp.npy...
/ [1 files][ 43.5 KiB/ 43.5 KiB]                                                
Operation completed over 1 objects/43.5 KiB.                                     
Copying gs://cs181_practical_data/Xtest_amp.npy...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .

## Imports

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import librosa

from torch import nn
from torchsummary import summary




### Load raw amplitude data as a numpy object.

In [6]:
# Load train data

X_amp_train = np.load("Xtrain_amp.npy")
y_amp_train = np.load("ytrain_amp.npy")

In [7]:
# Load test data

X_amp_test = np.load("Xtest_amp.npy")
y_amp_test = np.load("ytest_amp.npy")

### Load Mel spectrogram data as a numpy object.

In [8]:
# Load train data

X_mel_train = np.load("Xtrain_mel.npy")
y_mel_train = np.load("ytrain_mel.npy")

In [9]:
X_mel_train.shape

(5553, 128, 87)

In [10]:
# Flatten X_mel_train's spectrogram features
X_mel_train_flat = X_mel_train.reshape(X_mel_train.shape[0], -1)
X_mel_train_flat.shape

(5553, 11136)

In [11]:
# Load test data

X_mel_test = np.load("Xtest_mel.npy")
y_mel_test = np.load("ytest_mel.npy")

In [12]:
X_mel_test.shape

(2197, 128, 87)

In [13]:
# Flatten X_mel_test's spectrogram features
X_mel_test_flat = X_mel_test.reshape(X_mel_test.shape[0], -1)
X_mel_test_flat.shape

(2197, 11136)

## Metrics

In [14]:
'''
Calculate overall accuracy, and per-class accuracy
'''
def calc_metrics(model, is_raw_amp=True):
    if is_raw_amp:
        X_test = X_amp_test
        y_test = y_amp_test
        X_train = X_amp_train
        y_train = y_amp_train
    else:
        X_test = X_mel_test_flat
        y_test = y_mel_test
        X_train = X_mel_train_flat
        y_train = y_mel_train

    # Calculate overall accuracy
    test_acc_overall = model.score(X_test, y_test)
    train_acc_overall = model.score(X_train, y_train)

    # Calculate per-class accuracy
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)

   # Calculate accuracy for each dataset using confusion matrix
    cm_test = confusion_matrix(y_test, y_pred_test)
    cm_train = confusion_matrix(y_train, y_pred_train)

    test_acc_per_class = cm_test.diagonal()/cm_test.sum(axis=1)
    train_acc_per_class = cm_train.diagonal()/cm_train.sum(axis=1)

    classes = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']

    # Display metrics using classes as labels

    print("Overall accuracy on test set: {0:.2%}".format(test_acc_overall))
    print("Overall accuracy on train set: {0:.2%}".format(train_acc_overall))

    print("\nPer-class accuracy on test set:")
    for i in range(len(classes)):
        print("{0}: {1:.2%}".format(classes[i], test_acc_per_class[i]))

    print("\nPer-class accuracy on train set:")
    for i in range(len(classes)):
        print("{0}: {1:.2%}".format(classes[i], train_acc_per_class[i]))
        


## Part A: Feature Engineering and Baseline Models

### Raw Amplitude features

In [15]:
lrc_raw_2 = LogisticRegression(penalty='l2', tol = 10**-4, max_iter=1000)
lrc_raw_2.fit(X_amp_train, y_amp_train)

LogisticRegression(max_iter=1000)

In [13]:
lrc_raw = LogisticRegression(penalty='l2', tol = 10**-3, max_iter=1000)
lrc_raw.fit(X_amp_train, y_amp_train)

LogisticRegression(max_iter=1000, tol=0.001)

In [16]:
calc_metrics(lrc_raw_2, is_raw_amp=True)

Overall accuracy on test set: 17.89%
Overall accuracy on train set: 97.73%

Per-class accuracy on test set:
air_conditioner: 28.67%
car_horn: 0.00%
children_playing: 34.45%
dog_bark: 12.23%
drilling: 1.14%
engine_idling: 30.68%
gun_shot: 6.67%
jackhammer: 4.24%
siren: 13.98%
street_music: 15.67%

Per-class accuracy on train set:
air_conditioner: 99.29%
car_horn: 98.98%
children_playing: 98.13%
dog_bark: 95.03%
drilling: 99.01%
engine_idling: 97.64%
gun_shot: 97.59%
jackhammer: 99.54%
siren: 94.16%
street_music: 98.14%


In [14]:
calc_metrics(lrc_raw, is_raw_amp=True)

Overall accuracy on test set: 17.89%
Overall accuracy on train set: 97.73%

Per-class accuracy on test set:
air_conditioner: 28.67%
car_horn: 0.00%
children_playing: 34.45%
dog_bark: 12.23%
drilling: 1.14%
engine_idling: 30.68%
gun_shot: 6.67%
jackhammer: 4.24%
siren: 13.98%
street_music: 15.67%

Per-class accuracy on train set:
air_conditioner: 99.29%
car_horn: 98.98%
children_playing: 98.13%
dog_bark: 95.03%
drilling: 99.01%
engine_idling: 97.64%
gun_shot: 97.59%
jackhammer: 99.54%
siren: 94.16%
street_music: 98.14%


### Mel Spectogram Features

In [18]:
lrc_mel_2 = LogisticRegression(tol = 10**-3, max_iter=1000)
lrc_mel_2.fit(X_mel_train_flat, y_mel_train)

/Users/adammo/Desktop/cs181-env/cs181-s23-homeworks/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=0, tol=0.001)

In [19]:
lrc_mel = LogisticRegression(tol = 10**-4, max_iter=1000)
lrc_mel.fit(X_mel_train_flat, y_mel_train)

/Users/adammo/Desktop/cs181-env/cs181-s23-homeworks/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=0)

In [20]:
calc_metrics(lrc_mel, is_raw_amp=False)

Overall accuracy on test set: 36.69%
Overall accuracy on train set: 97.55%

Per-class accuracy on test set:
air_conditioner: 28.67%
car_horn: 33.33%
children_playing: 23.08%
dog_bark: 19.21%
drilling: 45.83%
engine_idling: 40.53%
gun_shot: 70.00%
jackhammer: 46.19%
siren: 70.34%
street_music: 23.33%

Per-class accuracy on train set:
air_conditioner: 97.71%
car_horn: 98.98%
children_playing: 95.98%
dog_bark: 94.46%
drilling: 98.85%
engine_idling: 95.98%
gun_shot: 100.00%
jackhammer: 99.54%
siren: 99.55%
street_music: 97.29%


## Part B: More Modeling & Hyperparameter Tuning and Validation

### Random Forest Classifier

We will instantiate and train an instance of the RandomForestClassifier class, with the number of estimators left at the default value of 100.

#### Using raw amplitude features:

In [15]:
# Import random forest classifier and k-nearest neighbors classifier
from sklearn.ensemble import RandomForestClassifier

# Create random forest classifier
rfc_raw = RandomForestClassifier()
rfc_raw.fit(X_amp_train, y_amp_train)

calc_metrics(rfc_raw, is_raw_amp=True)

Overall accuracy on test set: 24.62%
Overall accuracy on train set: 100.00%

Per-class accuracy on test set:
air_conditioner: 15.33%
car_horn: 0.00%
children_playing: 54.18%
dog_bark: 29.26%
drilling: 19.32%
engine_idling: 36.74%
gun_shot: 0.00%
jackhammer: 27.97%
siren: 8.05%
street_music: 11.00%

Per-class accuracy on train set:
air_conditioner: 100.00%
car_horn: 100.00%
children_playing: 100.00%
dog_bark: 100.00%
drilling: 100.00%
engine_idling: 100.00%
gun_shot: 100.00%
jackhammer: 100.00%
siren: 100.00%
street_music: 100.00%


#### Using Mel Spectogram features:

In [16]:
# Create random forest classifier
rfc_mel = RandomForestClassifier()
rfc_mel.fit(X_mel_train_flat, y_mel_train)

calc_metrics(rfc_mel, is_raw_amp=False)

Overall accuracy on test set: 49.20%
Overall accuracy on train set: 100.00%

Per-class accuracy on test set:
air_conditioner: 36.00%
car_horn: 25.64%
children_playing: 55.85%
dog_bark: 44.54%
drilling: 59.09%
engine_idling: 43.18%
gun_shot: 43.33%
jackhammer: 47.03%
siren: 61.86%
street_music: 51.33%

Per-class accuracy on train set:
air_conditioner: 100.00%
car_horn: 100.00%
children_playing: 100.00%
dog_bark: 100.00%
drilling: 100.00%
engine_idling: 100.00%
gun_shot: 100.00%
jackhammer: 100.00%
siren: 100.00%
street_music: 100.00%


#### Tuning the RFF

#### *First Iteration*

In [20]:
from sklearn.model_selection import GridSearchCV

def rfc_random_search(X, y, param_range=range(20, 200, 20)):
    # Create the parameter grid based on the results of random search
    param_grid = {
        'bootstrap': [True],
        'n_estimators': param_range
    }

    # Create a based model
    rf = RandomForestClassifier()

    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                                n_jobs = -1, verbose = 2)

    # Fit the grid search to the data
    grid_search.fit(X, y)

    return grid_search.best_params_

# Run random search on raw amplitude data
print("Best parameters for raw amplitude data:")
print(rfc_random_search(X_amp_train, y_amp_train))

# Run random search on mel spectrogram data
print("\nBest parameters for mel spectrogram data:")
print(rfc_random_search(X_mel_train_flat, y_mel_train))


Best parameters for raw amplitude data:
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ....................bootstrap=True, n_estimators=20; total time= 2.2min
[CV] END ....................bootstrap=True, n_estimators=20; total time= 2.2min
[CV] END ....................bootstrap=True, n_estimators=20; total time= 2.3min
[CV] END ....................bootstrap=True, n_estimators=20; total time= 2.3min
[CV] END ....................bootstrap=True, n_estimators=20; total time= 2.3min
[CV] END ....................bootstrap=True, n_estimators=40; total time= 3.7min
[CV] END ....................bootstrap=True, n_estimators=40; total time= 3.7min
[CV] END ....................bootstrap=True, n_estimators=40; total time= 3.7min
[CV] END ....................bootstrap=True, n_estimators=40; total time= 2.5min
[CV] END ....................bootstrap=True, n_estimators=40; total time= 2.5min
[CV] END ....................bootstrap=True, n_estimators=60; total time= 3.4min
[CV] END 

#### *More specific tuning*

In [28]:
# Run more specific random search on raw amplitude data

print("Best parameters for raw amplitude data:")
print(rfc_random_search(X_amp_train, y_amp_train, param_range=range(160, 200, 5)))

print("\nBest parameters for mel spectrogram data:")
print(rfc_random_search(X_mel_train_flat, y_mel_train, param_range=range(160, 200, 5)))

Best parameters for raw amplitude data:
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END ...................bootstrap=True, n_estimators=160; total time= 4.2min
[CV] END ...................bootstrap=True, n_estimators=160; total time= 4.3min
[CV] END ...................bootstrap=True, n_estimators=160; total time= 4.3min
[CV] END ...................bootstrap=True, n_estimators=160; total time= 4.3min
[CV] END ...................bootstrap=True, n_estimators=160; total time= 4.3min
[CV] END ...................bootstrap=True, n_estimators=165; total time= 4.4min
[CV] END ...................bootstrap=True, n_estimators=165; total time= 4.5min
[CV] END ...................bootstrap=True, n_estimators=165; total time= 4.5min
[CV] END ...................bootstrap=True, n_estimators=165; total time= 3.8min
[CV] END ...................bootstrap=True, n_estimators=165; total time= 3.8min
[CV] END ...................bootstrap=True, n_estimators=170; total time= 3.9min
[CV] END 

### K-Nearest Neighbours

We will instantiate and train an instance of the KNeighbours class, with the number of neighbours left at the default value of 5.

#### Using Raw Amplitude Features

In [17]:
from sklearn.neighbors import KNeighborsClassifier

# Create k-nearest neighbors classifier
knn_raw = KNeighborsClassifier()
knn_raw.fit(X_amp_train, y_amp_train)

calc_metrics(knn_raw, is_raw_amp=True)

Overall accuracy on test set: 17.84%
Overall accuracy on train set: 27.30%

Per-class accuracy on test set:
air_conditioner: 27.00%
car_horn: 0.00%
children_playing: 16.05%
dog_bark: 10.92%
drilling: 0.00%
engine_idling: 14.77%
gun_shot: 3.33%
jackhammer: 0.00%
siren: 80.93%
street_music: 2.33%

Per-class accuracy on train set:
air_conditioner: 41.43%
car_horn: 8.12%
children_playing: 37.07%
dog_bark: 30.02%
drilling: 0.66%
engine_idling: 28.85%
gun_shot: 9.64%
jackhammer: 0.30%
siren: 75.15%
street_music: 10.14%


#### Using Mel Spectogram Features

In [18]:
knn_mel = KNeighborsClassifier()
knn_mel.fit(X_mel_train_flat, y_mel_train)

calc_metrics(knn_mel, is_raw_amp=False)

Overall accuracy on test set: 26.90%
Overall accuracy on train set: 59.54%

Per-class accuracy on test set:
air_conditioner: 16.67%
car_horn: 38.46%
children_playing: 24.08%
dog_bark: 16.59%
drilling: 31.44%
engine_idling: 32.95%
gun_shot: 53.33%
jackhammer: 63.56%
siren: 26.27%
street_music: 6.00%

Per-class accuracy on train set:
air_conditioner: 78.29%
car_horn: 50.25%
children_playing: 46.84%
dog_bark: 41.11%
drilling: 67.22%
engine_idling: 87.93%
gun_shot: 66.27%
jackhammer: 82.37%
siren: 49.55%
street_music: 21.14%


#### Tuning KNN

In [26]:
from sklearn.model_selection import GridSearchCV

def knn_random_search(X, y, param_range=range(2, 10)):
    # Create the parameter grid based on the results of random search
    param_grid = {
        'n_neighbors': param_range
    }

    # Create a based model
    knn = KNeighborsClassifier()

    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = knn, param_grid = param_grid,
                                n_jobs = -1, verbose = 3)

    # Fit the grid search to the data
    grid_search.fit(X, y)

    return grid_search.best_params_

# Run random search on raw amplitude data
print("Best parameters for raw amplitude data:")
print(knn_random_search(X_amp_train, y_amp_train))

# Run random search on mel spectrogram data
print("\nBest parameters for mel spectrogram data:")
print(knn_random_search(X_mel_train_flat, y_mel_train))

Best parameters for raw amplitude data:
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 4/5] END .....................n_neighbors=2;, score=0.158 total time=  35.4s
[CV 5/5] END .....................n_neighbors=2;, score=0.198 total time=  35.9s
[CV 2/5] END .....................n_neighbors=3;, score=0.185 total time=  35.9s
[CV 3/5] END .....................n_neighbors=3;, score=0.157 total time=  36.0s
[CV 3/5] END .....................n_neighbors=2;, score=0.161 total time=  37.8s
[CV 2/5] END .....................n_neighbors=2;, score=0.189 total time=  37.8s
[CV 1/5] END .....................n_neighbors=2;, score=0.185 total time=  38.0s
[CV 1/5] END .....................n_neighbors=3;, score=0.179 total time=  38.0s
[CV 4/5] END .....................n_neighbors=3;, score=0.176 total time=  33.0s
[CV 5/5] END .....................n_neighbors=3;, score=0.192 total time=  35.9s
[CV 1/5] END .....................n_neighbors=4;, score=0.174 total time=  36.0s
[CV 2/5] 

In [27]:
knn_mel = KNeighborsClassifier(n_neighbors=3)
knn_mel.fit(X_mel_train_flat, y_mel_train)

calc_metrics(knn_mel, is_raw_amp=False)

Overall accuracy on test set: 27.04%
Overall accuracy on train set: 67.15%

Per-class accuracy on test set:
air_conditioner: 18.67%
car_horn: 38.46%
children_playing: 26.42%
dog_bark: 18.34%
drilling: 31.06%
engine_idling: 29.17%
gun_shot: 50.00%
jackhammer: 61.02%
siren: 27.12%
street_music: 6.67%

Per-class accuracy on train set:
air_conditioner: 84.14%
car_horn: 75.63%
children_playing: 61.78%
dog_bark: 50.10%
drilling: 72.82%
engine_idling: 90.43%
gun_shot: 71.08%
jackhammer: 86.17%
siren: 56.89%
street_music: 28.43%
